## Librerias y cpd-cli

In [ ]:
#!pip install kubernetes
#!pip install pandas
#!pip install python-dotenv


In [ ]:
#!wget https://github.com/IBM/cpd-cli/releases/download/v13.0.7/cpd-cli-linux-EE-13.1.7.tgz
#!tar zxf cpd-cli-linux-EE-13.1.7.tgz
#!rm cpd-cli-linux-EE-13.1.7.tgz

In [8]:
from kubernetes import client, config
from kubernetes.stream import stream
import pandas as pd
import re
import subprocess
import requests
import urllib3
import os
import json
from dotenv import load_dotenv
#deshabilita alertas
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

## Configuración cpd-cli

In [4]:
import os
import subprocess

# Load .env file
load_dotenv("config.env")

# Leer variables de entorno de archivo
api_key = os.getenv("API_KEY")
cpd_username = os.getenv("CPD_USERNAME")
local_user = os.getenv("LOCAL_USER")
cpd_profile_name = os.getenv("CPD_PROFILE_NAME")
cpd_profile_url = os.getenv("CPD_PROFILE_URL")



# Exportar variables de entorno
os.environ["API_KEY"] = api_key
os.environ["CPD_USERNAME"] = cpd_username
os.environ["LOCAL_USER"] = local_user
os.environ["CPD_PROFILE_NAME"] = cpd_profile_name
os.environ["CPD_PROFILE_URL"] = cpd_profile_url


# Execute the `cpd-cli` commands
try:
    subprocess.run(
        [
            "./cpd-cli/cpd-cli", "config", "users", "set", local_user,
            "--username", cpd_username,
            "--apikey", api_key
        ],
        check=True
    )

    subprocess.run(
        [
            "./cpd-cli/cpd-cli", "config", "profiles", "set", cpd_profile_name,
            "--user", local_user,
            "--url", cpd_profile_url
        ],
        check=True
    )

    print("configuración de cpd-cli finalizo con exito.")

except subprocess.CalledProcessError as e:
    print(f"Error ejecutando cpd-cli: {e}")


configuración de cpd-cli finalizo con exito.


## Configuración kubeconfig

In [5]:
# Cargar kubeconfig
config.load_kube_config("mlops_kubeconfig.yaml")
core_v1_api = client.CoreV1Api()

cpd_instance_url = cpd_profile_url  
os.environ['cpd_url']=cpd_instance_url
username = 'jlara'  # Reemplazar
namespace = "cpd"  # Reemplazar
label_selector = "app=asset-files-api"
base_paths = [
    "/mnt/asset_file_api/projects",
    "/mnt/asset_file_api/spaces"
]
# Temporales y salida final
output_file_intermediate = "file_sizes.csv"
output_file_with_names = "file_sizes_with_names.csv"
output_file_final = "data_proyectos.csv"
output_file_space_deployments = "space_deployments.csv"  # For spaces

## Obtencion data de proyectos

In [6]:
def get_projects_data(bearer_token):
    zen_watchdog_url = f"{cpd_instance_url}/zen-watchdog/v3/resources/workloads/policies/pretty/projects"
    headers = {
        'Authorization': f'Bearer {bearer_token}',
        'Accept': 'application/json'
    }
    response = requests.get(zen_watchdog_url, headers=headers, verify=False)
    response.raise_for_status()
    return response.json()

project_data = get_projects_data(os.environ['USER_ACCESS_TOKEN'])
projects = [
    {"ID": item["ID"], "Name": item["Name"]}
    for item in project_data["requestObj"]["Items"]
]
projects

In [26]:
print(cpd_instance_url)

https://cpd-cpd.apps.cp4d-education-install.cp.fyre.ibm.com/


In [29]:
print(os.environ['USER_ACCESS_TOKEN'])

eyJhbGciOiJSUzI1NiIsInR5cCI6IkpXVCIsImtpZCI6InFsYVZYd3VHdk5JWmQ4Z3dweTRZQzRXVkQzY3I0VURTRko0RmhtY3VsLWsifQ.eyJ1aWQiOiIxMDAwMzMxMDA1IiwidXNlcm5hbWUiOiJqbGFyYSIsInJvbGUiOiJBZG1pbiIsInBlcm1pc3Npb25zIjpbImNyZWF0ZV9wcm9qZWN0IiwiYWRtaW5pc3RyYXRvciIsImNhbl9wcm92aXNpb24iLCJtb25pdG9yX3BsYXRmb3JtIiwiY29uZmlndXJlX3BsYXRmb3JtIiwidmlld19wbGF0Zm9ybV9oZWFsdGgiLCJjb25maWd1cmVfYXV0aCIsIm1hbmFnZV91c2VycyIsIm1hbmFnZV9ncm91cHMiLCJtYW5hZ2Vfc2VydmljZV9pbnN0YW5jZXMiLCJtYW5hZ2VfY2F0YWxvZyIsIm1hbmFnZV92YXVsdHNfYW5kX3NlY3JldHMiLCJzaGFyZV9zZWNyZXRzIiwiYWRkX3ZhdWx0cyIsImNyZWF0ZV9zcGFjZSJdLCJncm91cHMiOlsxMDAwMF0sInN1YiI6ImpsYXJhIiwiaXNzIjoiS05PWFNTTyIsImF1ZCI6IkRTWCIsImFwaV9yZXF1ZXN0Ijp0cnVlLCJpYXQiOjE3MzgyNTE2NTQsImV4cCI6NTMzODI0ODA1NH0.Bnd6Ssic6MwJaE048oCDyEbxCq5IEDz1MSAcbEHNIMzjyx3MNwHSCzAzxyqrWZYlABFTmpYVvwxcW0vilEVb6I1lSA8PdMdpoCoyfqoJ0yqZ5fUxNYHeByYlFgr2hKjSfCGq4Gy0wfatnP5vmdTuCiAuzFxOHt6HzMn5Mo9mCv9FmPVQLVwjE19GILJP0OHwHckFc06MpJ_mykjsd4pgrhhIVFP29chC_Adgb-ubZWrUZ-gVhBhTjd2a_yQjc_rQZrOEDtt139FBx_KPunCSxG6sX

In [31]:
def get_space_data(bearer_token):
    zen_watchdog_url = f"{cpd_instance_url}#/Spaces/spaces_list"
    headers = {
        'Authorization': f"Bearer {bearer_token}",
        'Accept': 'application/json'
    }
    response = requests.get(zen_watchdog_url, headers=headers, verify=False)
    response.raise_for_status()
    return response

space_data = get_space_data(os.environ['USER_ACCESS_TOKEN'])
print(space_data)
#spaces = [
#    {"ID": item["ID"], "Name": item["Name"]}
#    for item in space_data["requestObj"]["Items"]
#]
#spaces

<Response [200]>


In [33]:
space_data.text

'<!doctype html><html lang="en"><head> <style>body {margin: 0;}\n  .appContainer { display: none }\n  @keyframes rotate\n  {\n    0% {\n      -webkit-transform: rotate(0deg);\n      transform: rotate(0deg);\n    }\n    100% {\n      -webkit-transform: rotate(360deg);\n      transform: rotate(360deg);\n    }\n  }\n  @keyframes skeleton {\n    0% { width:0; left:0; right:auto; opacity:.3;}\n    20% { width:100%; left:0; right:auto; opacity:1;}\n    28% { width:100%;left:auto;right:0;}\n    51% { width:0; left:auto; right:0;}\n    58% { width:0; left:auto; right:0;}\n    82% { width:100%; left:auto; right:0;}\n    83% { width:100%; left:0; right:auto;}\n    96%{ width:0; left:0; right:auto;}\n    to { width:0; left:0; right:auto; opacity:.3;}\n  }\n  .outline .appContainer { display: -ms-flexbox; display: flex; -ms-flex-direction: column; flex-direction: column; height: 100%; min-height: calc(100vh - 48px); overflow: auto; background-color: #f9f9f9; overflow: hidden;}\n  .outline .navigat

In [17]:
!cpd-cli/cpd-cli space list --profile mlops-profile

...
Nothing to show.


In [ ]:
!https://cpd-cpd.apps.cp4d-education-install.cp.fyre.ibm.com/ml-runtime/spaces/

In [ ]:
def is_valid_numeric_id(value):
    """Valida si el id es numerico"""
    return value.isdigit()

In [ ]:
# Obtener username en base a user_id
def get_username(user_id, bearer_token):
    user_mgmt_url = f"{cpd_instance_url}/usermgmt/v1/usermgmt/user/{user_id}?any_status=true&include_session_info=true"
    headers = {'Authorization': f'Bearer {bearer_token}', 'Accept': 'application/json'}
    response = requests.get(user_mgmt_url, headers=headers, verify=False)
    response.raise_for_status()
    user_data = response.json()
    return user_data.get('username', 'Unknown')

In [11]:
    pods = core_v1_api.list_namespaced_pod(namespace, label_selector=label_selector)
    if not pods.items:
        raise RuntimeError("No se encontro el pod")

    pod_name = pods.items[0].metadata.name
    print(f"Usando pod: {pod_name}")

Using pod: asset-files-api-769ff7cbcd-zf7m6


In [ ]:
# Main logic
try:
    # Find the pod
    pods = core_v1_api.list_namespaced_pod(namespace, label_selector=label_selector)
    if not pods.items:
        raise RuntimeError("No asset-files-api pod found.")

    pod_name = pods.items[0].metadata.name
    print(f"Using pod: {pod_name}")

    all_file_details = []
    for path in base_paths:
        print(f"Processing base path: {path}")

        # Command to list files and their sizes
        command = [
            "bash", "-c",
            f"shopt -s nullglob; for d in {path}/*/*/* {path}/*/*/.[!.]*; do if [ -e \"$d\" ]; then du -sh \"$d\"; fi; done"
        ]

        # Ejecutar en pod
        response = stream(
            core_v1_api.connect_get_namespaced_pod_exec,
            pod_name,
            namespace,
            command=command,
            stderr=True, stdin=False, stdout=True, tty=False,
        )

        # Parse response
        for line in response.splitlines():
            match = re.match(r"^(\S+)\s+(.+)$", line)
            if match:
                file_size, file_path = match.groups()
                parts = file_path.split("/")
                if "projects" in path and len(parts) >= 6:
                    project_id = parts[4]
                    user_id = parts[5]
                    if is_valid_numeric_id(user_id):
                        print(f"Appending valid user_id for projects: {user_id}")
                        file_name = parts[-1]
                        all_file_details.append({
                            "type": "project",
                            "id": project_id,
                            "user_id": user_id,
                            "file_name": file_name,
                            "file_size": file_size,
                        })
                    else:
                        print(f"Invalid user_id detected for projects: {user_id}, skipping.")
                elif "spaces" in path and len(parts) >= 5:
                    space_id = parts[4]
                    user_id = parts[5]
                    print(f"appending this user id for spaces: {user_id}")
                    file_name = parts[-1]
                    all_file_details.append({
                        "type": "space",
                        "id": space_id,
                        "user_id": user_id,
                        "file_name": file_name,
                        "file_size": file_size,
                    })

    df = pd.DataFrame(all_file_details)
    print(f"Collected {len(df)} file records.")
    df.to_csv(output_file_intermediate, index=False)
    print(f"Intermediate data saved to {output_file_intermediate}")

    # Add names for projects and spaces
    df["name"] = df.apply(
        lambda row: get_project_name(row["id"]) if row["type"] == "project" else get_space_name(row["id"]),
        axis=1
    )

    # Add usernames
    bearer_token = get_bearer_token()
    df["username"] = df["user_id"].apply(lambda user_id: get_username(user_id, bearer_token))

    # Save final output
    df.to_csv(output_file_final, index=False)
    print(f"Final data saved to {output_file_final}")

except Exception as e:
    print(f"An error occurred: {e}")